In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Loading in data from text file and export options

This should be the only part of the file that you need to change
1) The path to the data file
2) The number of samples total in the file 
3) the export name and path for the new datafile
4) If  you want to export a bar plot for each compound

In [2]:
file_path = 'Example files/20240606 PCBOCP EN651 GFF PUF.txt'
n_smp = 100
output_path = 'EN651 PCBOCP waters export.xlsx'
export_figures = True

Creating fuctions to extract the compound names and then removing the "compound: " bit

In [3]:
def read_specific_lines(file_path):
    result = []
    with open(file_path, 'r') as file:
        for i, line in enumerate(file, start=1):
            if (i - 5) % (n_smp + 4) == 0:
                result.append(line.strip())
    return result

def isolate_compound_name(compound_list):
    result = []
    for compound in compound_list:
        # Find the first space and take the substring from that point onward
        result.append(compound.split(' ', 2)[2].strip())
    return result

In [4]:
compounds = read_specific_lines(file_path)
compounds = isolate_compound_name(compounds)

Now there is a for loop that extract all the differnt tables from the text time and adds them together

In [12]:
data = pd.DataFrame()

lines = 6  # Start reading after the first 6 lines for the first table
for i in range(len(compounds)):
    if i == 0:
        data = pd.read_table(file_path, skiprows=lines, nrows=n_smp)
        data['Compound'] = compounds[i]
        lines += n_smp + 3  # +3 for the 3 blank lines
    else:
        data_temp = pd.read_table(file_path, skiprows=lines, nrows=n_smp)
        data_temp['Compound'] = compounds[i]
        data = pd.concat([data, data_temp], ignore_index=True, sort=False).reset_index(drop=True)
        lines += n_smp + 4  # +4 because we skip 3 blank lines and one header line


# Drop unwanted columns if they exist in the DataFrame
if 'Unnamed: 0' in data.columns or '#' in data.columns:
    data.drop(columns=['Unnamed: 0', '#'], inplace=True)

Now I fix the mistake where I didn't change the name of the sample for PAHs to PCBs/OCPs

In [13]:
data['Sample Text'] = data['Sample Text'].str.replace('PAHs','PCBs/OCPs')

Next I create different datafiles for each of the sets of compounds to export to excel 

In [14]:
inj_std = data.loc[data['Compound']=='RS TriBB (2,4,6-Tribromobiphenyl)']
IS_list = ['13C12 PCB-8','13C12 PCB-28','13C12 PCB-52','13C12 PCB-81','13C12 PCB-118','13C12 PCB-138','13C12 PCB-180',
           '13C12 PCB-209', '13C6-HCBz','13C12-DDT']
int_std = data.loc[data['Compound'].isin(IS_list)]
natives = data.loc[~data['Compound'].isin(IS_list)]

# Saving to excel

In [ ]:
position = output_path.find('waters')
output_path_with_natives = output_path[:position] + 'natives ' + output_path[position:]

In [1]:
data.to_excel(output_path,index=False)
natives.to_excel(output_path_with_natives,index=False)

NameError: name 'data' is not defined

# Save Figures

Here we save a plot of each of the compounds to look a trends across the sample set

In [16]:
def create_graphs(input_file):
    # Read the Excel file
    df = input_file

    # Create a folder to store the graphs if it doesn't exist
    output_folder = 'waters_export_compounds'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Group by Compound
    grouped = df.groupby('Compound')

    # Create and save graphs for each compound
    for compound, group in grouped:
        compound = compound.replace("/", "_")
        plt.figure(figsize=(15, 8))
        for id_ in group['ID'].unique():
            data = group[group['ID'] == id_]
            plt.bar(data['ID'], data['pg'], label=f"ID {id_}")
        plt.title(f'{compound}')
        plt.xlabel('ID')
        plt.ylabel('pg')
        plt.grid(True)
        plt.xticks(range(len(group['ID'].unique())), group['ID'].unique(),rotation=45,fontsize=8, ha='right')
        plt.tick_params(axis='x', which='major', pad=.5 )
        plt.gca().margins(x=0.05)
        plt.tight_layout()
        graph_filename = f"{compound}.png"
        plt.savefig(os.path.join(output_folder, graph_filename))
        plt.close()

In [ ]:
if export_figures == True
    if __name__ == "__main__":
        create_graphs(data)